In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
from datetime import datetime
import imageio
from PIL import Image

In [33]:
# Example usage # random generate graph with probability
G = nx.gnp_random_graph(100, 0.3)

In [36]:
def mis_p(G, start_number, full_output=True):
    step_counter = [start_number - 1]  # Tracks the current step number for output purposes
    total_steps = 0  # Counts the total number of recursive steps taken
    line10_calls = 0  # Counts how many times the general case (max degree node) is executed

    # Handle the base case: Empty graph
    if len(G) == 0:
        if full_output:
            print("The graph is empty. MIS size is 0.")  # Corresponds to an implicit check for an empty graph scenario
        return 0, total_steps, line10_calls

    if full_output:
        print("Starting the MIS algorithm with the original graph.")  # Introduction print statement for the start

    # Check if the maximum degree is ≤ 2 (Pseudocode lines 1-2)
    max_degree = max(dict(G.degree()).values())
    if max_degree <= 2:
        mis_set = nx.maximal_independent_set(G)  # Efficiently find MIS when max degree is ≤ 2
        if full_output:
            print(f"Found MIS of size {len(mis_set)}. (Line 1-2) When the maximum degree is 2 or less, we can efficiently compute the MIS.")
        return len(mis_set), total_steps, line10_calls

    # Find nodes with degree 1 (Pseudocode lines 3-4)
    degree_one_nodes = [n for n, d in G.degree() if d == 1]
    if degree_one_nodes:
        v = degree_one_nodes[0]
        G_minus_N_v = G.copy()
        G_minus_N_v.remove_nodes_from(list(G.neighbors(v)) + [v])  # Remove v and its neighbors
        if full_output:
            print(f"Removed Node {v} (Degree 1) and Its Neighbors. (Line 3-4) This reduces the problem size.")
        sub_mis_size, sub_steps, sub_line10 = mis_p(G_minus_N_v, step_counter[0] + 1, full_output)
        total_steps += sub_steps
        return 1 + sub_mis_size, total_steps, line10_calls

    # Handle disconnected graphs (Pseudocode lines 5-7)
    if not nx.is_connected(G):
        components = list(nx.connected_components(G))
        G1 = G.subgraph(components[0])
        G_minus_G1 = G.copy()
        G_minus_G1.remove_nodes_from(components[0])
        if full_output:
            print("The graph is not connected. (Line 5-7) We solve MIS for each connected component separately.")
        sub_mis_size1, sub_steps1, sub_line10_1 = mis_p(G1, step_counter[0] + 1, full_output)
        sub_mis_size2, sub_steps2, sub_line10_2 = mis_p(G_minus_G1, step_counter[0] + sub_steps1 + 1, full_output)
        total_steps += sub_steps1 + sub_steps2
        return sub_mis_size1 + sub_mis_size2, total_steps, line10_calls

    # General case for a node with the maximum degree (Pseudocode lines 8-10)
    max_degree_node = max(G.degree(), key=lambda x: x[1])[0]
    G_minus_N_v = G.copy()
    G_minus_N_v.remove_nodes_from(list(G.neighbors(max_degree_node)) + [max_degree_node])
    G_minus_v = G.copy()
    G_minus_v.remove_node(max_degree_node)
    
    if full_output:
        print(f"Exploring cases with and without node {max_degree_node}. (Line 8-10)")
    
    line10_calls += 1
    sub_mis_size1, sub_steps1, sub_line10_1 = mis_p(G_minus_N_v, step_counter[0] + 1, full_output)
    sub_mis_size2, sub_steps2, sub_line10_2 = mis_p(G_minus_v, step_counter[0] + sub_steps1 + 1, full_output)
    total_steps += sub_steps1 + sub_steps2
    line10_calls += sub_line10_1 + sub_line10_2
    return max(1 + sub_mis_size1, sub_mis_size2), total_steps, line10_calls

# Example usage
G = nx.gnp_random_graph(15, 0.3)  # Example graph with 15 nodes and edge probability of 0.3
mis_size, total_steps, line10_calls = mis_p(G, 1)
print(f"MIS Size: {mis_size}, Total Steps: {total_steps}, Line 10 Calls: {line10_calls}")


Starting the MIS algorithm with the original graph.
Removed Node 8 (Degree 1) and Its Neighbors. (Line 3-4) This reduces the problem size.
Starting the MIS algorithm with the original graph.
Exploring cases with and without node 11. (Line 8-10)
Starting the MIS algorithm with the original graph.
Removed Node 2 (Degree 1) and Its Neighbors. (Line 3-4) This reduces the problem size.
Starting the MIS algorithm with the original graph.
Found MIS of size 2. (Line 1-2) When the maximum degree is 2 or less, we can efficiently compute the MIS.
Starting the MIS algorithm with the original graph.
Exploring cases with and without node 10. (Line 8-10)
Starting the MIS algorithm with the original graph.
Removed Node 0 (Degree 1) and Its Neighbors. (Line 3-4) This reduces the problem size.
Starting the MIS algorithm with the original graph.
Removed Node 6 (Degree 1) and Its Neighbors. (Line 3-4) This reduces the problem size.
Starting the MIS algorithm with the original graph.
Found MIS of size 1. (

In [93]:
nx.maximal_independent_set(G)

[4, 9, 14, 8, 2, 13]

In [26]:
def mis2(G):
    if G.number_of_nodes() == 0:
        return 0
    
    if max(dict(G.degree()).values()) <= 2:
        return len(nx.algorithms.approximation.maximum_independent_set(G))

    for v in G.nodes():
        if G.degree(v) == 1:
            neighbors = list(G[v])
            subgraph = G.copy()
            subgraph.remove_nodes_from(neighbors)
            subgraph.remove_node(v)
            return 1 + mis2(subgraph)

    if not nx.is_connected(G):
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        return sum(mis2(comp) for comp in components)

    max_degree = max(dict(G.degree()).values())
    candidates = [v for v, d in G.degree() if d == max_degree]
    best = 0
    for v in candidates:
        subgraph1 = G.copy()
        subgraph1.remove_nodes_from([v] + list(G.neighbors(v)))
        option1 = 1 + mis2(subgraph1)
        
        subgraph2 = G.copy()
        subgraph2.remove_node(v)
        option2 = mis2(subgraph2)
        
        best = max(best, option1, option2)
    
    return best


# Compute the maximum independent set size
max_independent_set_size = mis2(G)
print("Maximum Independent Set Size:", max_independent_set_size)


KeyboardInterrupt: 

In [ ]:
import networkx as nx

def mis(G):
    # Base case: check if the graph is empty, return 0 because an empty graph has no independent vertices
    if G.number_of_nodes() == 0:
        return 0
    
    # Step 1: Check if the maximum degree of the graph is at most 2
    # If true, compute and return the size of a maximum independent set (simplified here using an approximation algorithm)
    if max(dict(G.degree()).values()) <= 2:
        return len(nx.algorithms.approximation.maximum_independent_set(G))

    # Step 2: Check for any vertex with degree 1
    # If found, include this vertex in the MIS, remove it and its neighbors from the graph, and recurse
    for v in G.nodes():
        if G.degree(v) == 1:
            neighbors = list(G[v])
            subgraph = G.copy()
            subgraph.remove_nodes_from(neighbors)  # Remove neighbors
            subgraph.remove_node(v)  # Remove the vertex itself
            return 1 + mis(subgraph)  # Include this vertex in the independent set and recurse

    # Step 3: If the graph is not connected
    # Split the graph into connected components and compute the MIS for each component, summing the results
    if not nx.is_connected(G):
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        return sum(mis(comp) for comp in components)  # Sum the MIS of each component

    # Step 4: General case
    # Find a vertex with the maximum degree and consider two cases: including or excluding it from the MIS
    max_degree = max(dict(G.degree()).values())
    candidates = [v for v, d in G.degree() if d == max_degree]
    best = 0
    for v in candidates:
        # Option 1: Include this vertex in the MIS
        subgraph1 = G.copy()
        subgraph1.remove_nodes_from([v] + list(G.neighbors(v)))  # Remove v and its neighbors
        option1 = 1 + mis(subgraph1)  # Compute MIS for the remaining graph

        # Option 2: Exclude this vertex from the MIS
        subgraph2 = G.copy()
        subgraph2.remove_node(v)  # Remove only v
        option2 = mis(subgraph2)  # Compute MIS for the remaining graph

        # Select the maximum size of the independent set obtained from both options
        best = max(best, option1, option2)
    
    return best


print("Maximum Independent Set Size:", mis(G))  # Compute and print the maximum independent set size


Maximum Independent Set Size: 7
